In [24]:
import numpy as np
import pandas as pd
import pickle
import os 

import parsimony.functions.nesterov.tv as tv
from parsimony.estimators import LinearRegressionL1L2TV, ElasticNet
from parsimony.estimators import GridSearchKFoldRegression

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import squareform

from tv_utils import get_data_ij
from data_load import load_data, load_meshes_coor_tria


path = '/cobrain/groups/ml_group/data/HCP/HCP/'
connec , thinkness, log_jac, unique_labels, labels, mean_area_eq, idx_subj_connec = load_data(path)

# path_tria= "/cobrain/groups/ml_group/data/HCP/HCP/"
path_new_tria = '/home/bgutman/datasets/HCP/Simplified_templates/'

name_simp = '_200_mean_IC5.m'
# name_orig = '_200_mean.m' 

# orig_coord, orig_tria = load_meshes_coor_tria(path_tria, name_orig)
new_coord , new_tria = load_meshes_coor_tria(path_new_tria, name_simp)


path_res = '/home/ayagoz/connec/results/'
                   
Y = connec[:,0,:,:]


m = int(len(new_coord)/2)
new_log_jac = np.concatenate([log_jac[:,0, :m], log_jac[:,1,:m]], axis = 1)

new_thick = np.concatenate([thinkness[:,0, :m], thinkness[:,1,:m]], axis = 1)

new_area = np.concatenate([mean_area_eq[0, :m] * np.exp(new_log_jac[:,:m]), 
                           mean_area_eq[1, :m] * np.exp(new_log_jac[:,m:])], axis = 1)

new_labels = np.concatenate([labels[0,:m], labels[1,:m]], axis = 0)

print('new ', new_log_jac.shape, new_thick.shape, new_area.shape, new_labels.shape)


Connectomes:  (789, 2, 68, 68)
Thinkness:  (789, 2, 163842)
Log Jacob:  (789, 2, 163842)
Unique Labels:  (789, 2, 163842)
Mean Labels:  (2, 163842)
Mean Area eq:  (2, 163842)
Meshes coordinates shape:  (20484, 3)
Number of triangles of meshes:  (40960, 3)
new  (789, 20484) (789, 20484) (789, 20484) (20484,)


In [25]:
log_jac.shape, thinkness.shape, mesh_area.shape, mean_labels.shape

((789, 2, 163842), (789, 2, 163842), (789, 2, 163842), (2, 163842))

In [26]:
m = int(len(new_coord)/2)

new_log_jac = np.concatenate([log_jac[:,0, :m], log_jac[:,1,:m]], axis = 1)
new_thick = np.concatenate([thinkness[:,0, :m], thinkness[:,1,:m]], axis = 1)
new_area = np.concatenate([mesh_area[:,0, :m], mesh_area[:,1,:m]], axis = 1)
new_labels = np.concatenate([mean_labels[0,:m], mean_labels[1,:m]], axis = 0)
new_log_jac.shape, new_thick.shape, new_area.shape, new_labels.shape

((789, 20484), (789, 20484), (789, 20484), (20484,))

In [27]:
idx_nodes = list(range(1,4)) + list(range(5,39)) + list(range(40,71))
idx_nodes = np.array(idx_nodes)
print(idx_nodes.shape)

(68,)


In [28]:
i = 54
j = 62
coord_ij, tria_ij, data_ij  = get_data_ij(idx_nodes[i], idx_nodes[j], new_labels, new_tria, new_coord, new_log_jac)
X = data_ij
X_train, X_test, y_train, y_test = train_test_split(X, Y[:,i,j], shuffle = True,
                                                        random_state = 1, test_size = 0.33)

In [29]:
coord_ij.shape, tria_ij.shape, data_ij.shape

((1003, 3), (1759, 3), (789, 1003))

In [33]:
A_ij = tv.linear_operator_from_mesh(coord_ij, tria_ij)

l1 = [0.1, 0.3, 0.5, 0.8, 1., 3.]
l2 = [0., 0.1, 0.3]
tv = [0.1, 0.5, 1., 3., 5.]


# DEFINE FUNCTION THAT CHOOSE EDGES AND MAKE LIST OF TUPLES (i,j)

### AAAAAAAAAAAAAAAAAA

SMTH LIKE:

    - the most different edges
    - the variableness or smth
    - the best coherense or smth like this (согласованные)

In [ ]:
results = pd.DataFrame(columns=['edge', 'l1','l2','tv', 'mse_test', 'r2_test'])
row = 0
for e in edges:
    for a in l1:
        for b in l2:
            for c in tv:
                i, j = e
                coord_ij, tria_ij, data_ij  = get_data_ij(idx_nodes[i], idx_nodes[j], 
                                                          new_labels, new_tria, new_coord, new_log_jac)
                X = data_ij
                X_train, X_test, y_train, y_test = train_test_split(X, Y[:,i,j], shuffle = True,
                                                                        random_state = 1, test_size = 0.33)
                
                tv_reg = LinearRegressionL1L2TV(**{'l1':a, 'l2':b, 'tv':c}, A = A_ij)
                tv_reg.fit(X_train, y_train)
                
                y_train_pred = tv_reg.predict(X_train)
                y_test_pred = tv_reg.predict(X_test)
                
                mse_test = mean_squared_error(y_test, y_test_pred)
                r2_test = r2_score(y_test, y_test_pred_tv)
                print('TRAIN MSE TV: {}, TEST MSE TV: {}'.format(mean_squared_error(y_train, y_train_pred),
                                                                mse_test))

                print('TRAIN R2 TV: {}, TEST R2 TV: {}\n'.format(r2_score(y_train, y_train_pred),
                                                                r2_test))
                results.loc[row] = [e, a, b, c, mse_test, r2_test]
    row += 1

In [ ]:
results.to_csv('/home/ayagoz/connec/results/TV/all_IC5_edge.csv')

In [ ]:

params_l1l2tv = {'l1': 0.9, 'l2':0.0, 'tv':.9}
params_l1l2 = {'l': 1., 'alpha':0.9}

# l in params_l1l2 refers to l1 in paramss_l1l2tv,
# l2 from params_l1l2tv = alpha * (1 - l) / 2
l = [0.1, 0.3, 0.5, 0.8, 1., 3.]
alpha = [1.]
reg = ElasticNet(**params_l1l2)

In [34]:
tv_reg.set_params(**{'l1':0.1, 'l2':.0, 'tv': 0.1})

In [39]:
tv_reg.get_params()

{'A': [<1003x1003 sparse matrix of type '<class 'numpy.float64'>'
  	with 1890 stored elements in Compressed Sparse Row format>,
  <1003x1003 sparse matrix of type '<class 'numpy.float64'>'
  	with 1710 stored elements in Compressed Sparse Row format>,
  <1003x1003 sparse matrix of type '<class 'numpy.float64'>'
  	with 1498 stored elements in Compressed Sparse Row format>,
  <1003x1003 sparse matrix of type '<class 'numpy.float64'>'
  	with 310 stored elements in Compressed Sparse Row format>,
  <1003x1003 sparse matrix of type '<class 'numpy.float64'>'
  	with 96 stored elements in Compressed Sparse Row format>,
  <1003x1003 sparse matrix of type '<class 'numpy.float64'>'
  	with 16 stored elements in Compressed Sparse Row format>],
 'l1': 0.1,
 'l2': 0.0,
 'mean': True,
 'mu': 5e-08,
 'penalty_start': 0,
 'rho': 1.0,
 'tv': 0.1}

In [16]:
tv_reg.fit(X_train, y_train)
y_pred_tv = tv_reg.predict(X_test, )

reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

y_train_pred = reg.predict(X_train)

print('TRAIN MSE TV: {}, TEST MSE TV: {}'.format(mean_squared_error(y_train, y_train_pred_tv),
                                                mean_squared_error(y_test, y_pred_tv)))

print('TRAIN R2 TV: {}, TEST R2 TV: {}\n'.format(r2_score(y_train, y_train_pred_tv),
                                                r2_score(y_test, y_pred_tv)))

print('TRAIN MSE regression: {}, TEST MSE regression: {}'.format(mean_squared_error(y_train, y_train_pred),
                                                mean_squared_error(y_test, y_pred)))

print('TRAIN R2 regression: {}, TEST R2 regression: {}'.format(r2_score(y_train, y_train_pred),
                                                r2_score(y_test, y_pred)))

KeyboardInterrupt: 